In [1]:
# Test
file_name="./aoc_2115_test.txt"
# Prod
file_name="./aoc_2115_prod.txt"

In [2]:
with open(file_name) as f:
    raw_data=[line for line in f.read().splitlines()]
    f.close()

# Check
display(raw_data[:10])

['6489887186259823155119378351872148719812918213519649577918412241959623221389974192429139298912991367',
 '3859891261917894749518659872199265294116953513169391165342899913878588561865553734597561693917577259',
 '7821697449796196228941887781377133998193483644892699199569294366179448944296919898443894894653982299',
 '4521784638899152252884535888651941499199556787174181866459996191939865591888468793671399848719386874',
 '9191996931799877154649198443239834219617959491499995891819688991189512152374148611392199689771321748',
 '3667131999966376698925485914885119653598637984789199249717987982323518981692239229913697228977769187',
 '5913911179874379588135157927159887689932919711124715316555782223929991889593659797141392198858861889',
 '8889299854191893897698362569935143896411522526753243281794876746461747369943293918696398639953911511',
 '6463567347538215129956611194361438953389822419896312246999991693527638996861829479689711567997995192',
 '49698122549967859711522458598913288716985541946134998

In [3]:
data=[[int(digit) for digit in line] for line in raw_data]

In [4]:
#data

## Part 0

In [5]:
from typing import Tuple, List

In [6]:
import pandas as pd
pd.__version__

import hana_ml
print(hana_ml.__version__)

2.10.21091803


In [7]:
hana_cloud_endpoint="402c4ef5-358b-4a67-b849-242c5ca7da89.hana.prod-eu10.hanacloud.ondemand.com:443"

In [8]:
hana_cloud_host, hana_cloud_port=hana_cloud_endpoint.split(":")

cchc=hana_ml.dataframe.ConnectionContext(port=hana_cloud_port,
                                         address=hana_cloud_host,
                                         user='DataGeek'
                                        )

HANA DB User : DataGeek Password :  ·········


In [9]:
print(cchc.hana_version())
print(cchc.get_current_schema())

4.00.000.00.1638878465 (fa/CE2021.30)
DATAGEEK


## Part 1

In [10]:
max_y=len(data)
max_x=len(data[0])
max_y, max_x

(100, 100)

In [11]:
%%time
nodes4dfp={
    'node_id':    [y*max_x+ x   for y in range(max_y) for x in range(max_x)],
    'node_coord': [(y,x)    for y in range(max_y) for x in range(max_x)],
    'node_name':  [str((y,x))    for y in range(max_y) for x in range(max_x)],
    'own_cost':   [data[y][x] for y in range(max_y) for x in range(max_x)]
}

CPU times: user 7.09 ms, sys: 515 µs, total: 7.61 ms
Wall time: 7.54 ms


In [12]:
dfp_nodes=pd.DataFrame(nodes4dfp)

# Check
dfp_nodes

,node_id,node_coord,node_name,own_cost
0,0,"(0, 0)","(0, 0)",6
1,1,"(0, 1)","(0, 1)",4
2,2,"(0, 2)","(0, 2)",8
3,3,"(0, 3)","(0, 3)",9
4,4,"(0, 4)","(0, 4)",8
...,...,...,...,...
9995,9995,"(99, 95)","(99, 95)",9
9996,9996,"(99, 96)","(99, 96)",9
9997,9997,"(99, 97)","(99, 97)",3
9998,9998,"(99, 98)","(99, 98)",1


In [13]:
def get_neighbours(yx: Tuple[int,int]) -> List[Tuple[int, int]]:
    y,x=yx
    neighbours=[
        (ny,nx) 
        for ny in range(max(0,y-1), min(y+2, max_y))
        for nx in range(max(0,x-1), min(x+2, max_x))
        if not(ny==y and nx==x) and not(ny!=y and nx!=x)
    ]
    return neighbours

#Check
get_neighbours((8,8))

[(7, 8), (8, 7), (8, 9), (9, 8)]

In [14]:
def get_neighbours_by_id(node_id: int) -> List[Tuple[int, int]]:
    y,x=dfp_nodes[dfp_nodes['node_id']==node_id]['node_coord'].iloc[0]
    neighbours_coord=[
        (ny,nx) 
        for ny in range(max(0,y-1), min(y+2, max_y))
        for nx in range(max(0,x-1), min(x+2, max_x))
        if not(ny==y and nx==x) and not(ny!=y and nx!=x)
    ]
    neighbours_ids_costs=[
        (dfp_nodes[dfp_nodes['node_coord']==neighbour_coord][['node_id','own_cost']].values[0][0], #node_id
        dfp_nodes[dfp_nodes['node_coord']==neighbour_coord][['node_id','own_cost']].values[0][1]) #node_cost
        for neighbour_coord in neighbours_coord
    ]
    return neighbours_ids_costs

#Check
get_neighbours_by_id(99)

[(98, 6), (199, 9)]

In [15]:
edges4dfp={
    'edge_id': [],
    'node_from': [],
    'node_to': [],
    'cost': []
}

edge_id=0
for node_from_id in nodes4dfp['node_id']:
    node_from_coord=dfp_nodes[dfp_nodes['node_id']==node_from_id]['node_coord'].iloc[0]
    for neighbour in get_neighbours_by_id(node_from_id):
        edge_id+=1
        #print(edge_id, node_from_id, neighbour[0], neighbour[1])
        edges4dfp['edge_id'].append(edge_id)
        edges4dfp['node_from'].append(node_from_id)
        edges4dfp['node_to'].append(neighbour[0])
        edges4dfp['cost'].append(float(neighbour[1]))

In [16]:
dfp_edges=pd.DataFrame(edges4dfp)

dfp_edges

,edge_id,node_from,node_to,cost
0,1,0,1,4.0
1,2,0,100,3.0
2,3,1,0,6.0
3,4,1,2,8.0
4,5,1,101,8.0
...,...,...,...,...
39595,39596,9998,9898,4.0
39596,39597,9998,9997,3.0
39597,39598,9998,9999,8.0
39598,39599,9999,9899,6.0


In [17]:
import hana_ml.graph

hg = hana_ml.graph.create_graph_from_dataframes(connection_context=cchc,
                                                vertices_df=dfp_nodes[['node_id','node_name','own_cost']].convert_dtypes(), 
                                                edges_df=dfp_edges,
                                                workspace_name="AOC2021DAY15",
                                                vertex_key_column="node_id",
                                                edge_key_column="edge_id", edge_source_column="node_from", edge_target_column="node_to",
                                                drop_exist_tab=True
                                               )

100%|█████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.76it/s]


In [18]:
print(cchc.sql("SELECT SCHEMA_NAME, TABLE_NAME FROM TABLES WHERE SCHEMA_NAME='{schema_name}'"
                 .format(schema_name=cchc.get_current_schema()))
        .collect()
       )

  SCHEMA_NAME             TABLE_NAME
0    DATAGEEK     AOC2021DAY15_EDGES
1    DATAGEEK  AOC2021DAY15_VERTICES


In [19]:
hana_ml.graph.discover_graph_workspaces(connection_context=cchc)

,SCHEMA_NAME,WORKSPACE_NAME,IS_VALID,EDGE_SCHEMA_NAME,EDGE_TABLE_NAME,EDGE_KEY_COLUMN_NAME,EDGE_SOURCE_COLUMN_NAME,EDGE_TARGET_COLUMN,VERTEX_SCHEMA_NAME,VERTEX_TABLE_NAME,VERTEX_KEY_COLUMN_NAME
0,DATAGEEK,AOC2021DAY15,TRUE,DATAGEEK,AOC2021DAY15_EDGES,edge_id,node_from,node_to,DATAGEEK,AOC2021DAY15_VERTICES,node_id


In [20]:
start_node=dfp_nodes[dfp_nodes['node_coord']==(0,0)][['node_id','own_cost']].values[0][0]
end_node=dfp_nodes[dfp_nodes['node_coord']==(max_y-1,max_x-1)][['node_id','own_cost']].values[0][0]
start_node, end_node

(0, 9999)

In [21]:
%%time
import hana_ml.graph.algorithms
sp = hana_ml.graph.algorithms.ShortestPath(graph=hg).execute(source=str(start_node), target=str(end_node), weight="cost")

CPU times: user 6.67 ms, sys: 4.82 ms, total: 11.5 ms
Wall time: 444 ms


In [22]:
result=int(sp.weight)

In [23]:
print(f"Result for part 1: {result}")

Result for part 1: 609


## Part 2

In [24]:
def next_digit(input: int)->int:
    return (input+1)%10 + (input+1) // 10

for i in range(1,10):
    print(i, next_digit(i))

1 2
2 3
3 4
4 5
5 6
6 7
7 8
8 9
9 1


In [25]:
MULTIPLIER=5
new_data=[]

for m_y in range(MULTIPLIER):
    for y in range(max_y):
        new_data.append(
            [next_digit(digit+m_y-1) for digit in data[y]]
        )

max_new_y=len(new_data)
print(max_new_y)

for y in range(max_new_y):
    for m_x in range(MULTIPLIER-1):
        new_data[y]+=[next_digit(digit+m_x) for digit in new_data[y][:max_x]]
max_new_x=len(new_data[0])
print(max_new_x)

500
500


In [26]:
from copy import deepcopy
data=deepcopy(new_data)

In [27]:
max_y=len(data)
max_x=len(data[0])
max_y, max_x

(500, 500)

In [28]:
%%time
nodes4dfp={
    'node_id':    [y*max_x+ x   for y in range(max_y) for x in range(max_x)],
    'node_coord': [(y,x)    for y in range(max_y) for x in range(max_x)],
    'node_name':  [str((y,x))    for y in range(max_y) for x in range(max_x)],
    'own_cost':   [data[y][x] for y in range(max_y) for x in range(max_x)]
}

CPU times: user 159 ms, sys: 12.4 ms, total: 171 ms
Wall time: 170 ms


In [29]:
dfp_nodes=pd.DataFrame(nodes4dfp)

# Check
dfp_nodes

,node_id,node_coord,node_name,own_cost
0,0,"(0, 0)","(0, 0)",6
1,1,"(0, 1)","(0, 1)",4
2,2,"(0, 2)","(0, 2)",8
3,3,"(0, 3)","(0, 3)",9
4,4,"(0, 4)","(0, 4)",8
...,...,...,...,...
249995,249995,"(499, 495)","(499, 495)",8
249996,249996,"(499, 496)","(499, 496)",8
249997,249997,"(499, 497)","(499, 497)",2
249998,249998,"(499, 498)","(499, 498)",9


In [30]:
%%time
edges4dfp={
    'edge_id': [],
    'node_from': [],
    'node_to': [],
    'cost': []
}

edge_id=0
for node_from_id in nodes4dfp['node_id']:
    node_from_coord=dfp_nodes[dfp_nodes['node_id']==node_from_id]['node_coord'].iloc[0]
    for neighbour in get_neighbours_by_id(node_from_id):
        edge_id+=1
        #print(edge_id, node_from_id, neighbour[0], neighbour[1])
        edges4dfp['edge_id'].append(edge_id)
        edges4dfp['node_from'].append(node_from_id)
        edges4dfp['node_to'].append(neighbour[0])
        edges4dfp['cost'].append(float(neighbour[1]))

CPU times: user 7h 5min 27s, sys: 18.6 s, total: 7h 5min 46s
Wall time: 7h 6min 1s


In [31]:
dfp_edges=pd.DataFrame(edges4dfp)

dfp_edges

,edge_id,node_from,node_to,cost
0,1,0,1,4.0
1,2,0,500,3.0
2,3,1,0,6.0
3,4,1,2,8.0
4,5,1,501,8.0
...,...,...,...,...
997995,997996,249998,249498,3.0
997996,997997,249998,249997,2.0
997997,997998,249998,249999,7.0
997998,997999,249999,249499,5.0


In [32]:
import hana_ml.graph

hg = hana_ml.graph.create_graph_from_dataframes(connection_context=cchc,
                                                vertices_df=dfp_nodes[['node_id','node_name','own_cost']].convert_dtypes(), 
                                                edges_df=dfp_edges,
                                                workspace_name="AOC2021DAY15",
                                                vertex_key_column="node_id",
                                                edge_key_column="edge_id", edge_source_column="node_from", edge_target_column="node_to",
                                                drop_exist_tab=True
                                               )

100%|███████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:16<00:00,  1.25it/s]


In [33]:
print(cchc.sql("SELECT SCHEMA_NAME, TABLE_NAME FROM TABLES WHERE SCHEMA_NAME='{schema_name}'"
                 .format(schema_name=cchc.get_current_schema()))
        .collect()
       )

  SCHEMA_NAME             TABLE_NAME
0    DATAGEEK     AOC2021DAY15_EDGES
1    DATAGEEK  AOC2021DAY15_VERTICES


In [34]:
hana_ml.graph.discover_graph_workspaces(connection_context=cchc)

,SCHEMA_NAME,WORKSPACE_NAME,IS_VALID,EDGE_SCHEMA_NAME,EDGE_TABLE_NAME,EDGE_KEY_COLUMN_NAME,EDGE_SOURCE_COLUMN_NAME,EDGE_TARGET_COLUMN,VERTEX_SCHEMA_NAME,VERTEX_TABLE_NAME,VERTEX_KEY_COLUMN_NAME
0,DATAGEEK,AOC2021DAY15,TRUE,DATAGEEK,AOC2021DAY15_EDGES,edge_id,node_from,node_to,DATAGEEK,AOC2021DAY15_VERTICES,node_id


In [35]:
start_node=dfp_nodes[dfp_nodes['node_coord']==(0,0)][['node_id','own_cost']].values[0][0]
end_node=dfp_nodes[dfp_nodes['node_coord']==(max_y-1,max_x-1)][['node_id','own_cost']].values[0][0]
start_node, end_node

(0, 249999)

In [36]:
%%time
import hana_ml.graph.algorithms
sp = hana_ml.graph.algorithms.ShortestPath(graph=hg).execute(source=str(start_node), target=str(end_node), weight="cost")

CPU times: user 6.29 ms, sys: 4.42 ms, total: 10.7 ms
Wall time: 1.51 s


In [37]:
result=int(sp.weight)

In [38]:
print(f"Result for part 2: {result}")

Result for part 2: 2925
